### recipe_ingredients 전처리 (시험)
1. 이상한 문자열 제거(uf2ff, u200b)
2. 식재료 칼럼 만들기(ingrdient, quantity, unit)
3. recipe_ingredients 칼럼을 식재료 칼럼으로 나누기
   - (정규표현식: '([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?')
   - 1000개에 3초 (-> 11만개 330초 정도)
4. 어떤 단위가 있는지 확인

논의할 부분
1. 개, 컵, 줄 등은 식재료에 따라서 용량이 달라지는 문제
2. 용량이 없거나, 약간, 조금처럼 표현하는 것을 어떻게 처리할지 -> (일단은 median or 최빈값으로 대체)

In [8]:
import oracledb as od
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn= od.connect(user='admin', password='INISW2inisw2', dsn='inisw2_high')
cursor = conn.cursor()
aa = cursor.execute('SELECT * FROM RECIPE_DATA_TABLE where recipe_comment = 11')
rows = cursor.fetchall()
conn.close()

('http://www.10000recipe.com/recipe/6869920', '입맛돋궈주는 새콤아삭 콩나물냉채', '4인분', '15분 이내', '아무나', "{'재료': ['콩나물200g', '맛살4T', '당근조금', '양파1/4개', '오이(생략가능)1/2개'], '소스': ['설탕4.5T', '다진마늘0.5T', '간장2T', '식초5T', '매실액기스(올리고당대체가능)1T', '참기름쪼르륵', '깨솔솔', '겨자1.5T']}", "['입맛돋우는음식', '간단반찬', '새콤달콤반찬', '밑반찬', '콩나물냉채']", 11, 104000, '샐러드', '무침', '일상', <oracledb.LOB object at 0x0000019A60DD2550>, None)
('http://www.10000recipe.com/recipe/1673173', '고구마줄기볶음', '6인분 이상', '15분 이내', '아무나', "{'재료': ['고구마줄기1줌'], '양념': ['참기름2스푼', '다진마늘1/2스푼', '진간장3스푼', '소금1스푼', '요리당1스푼', '볶은깨1/2스푼']}", '[]', 11, 13000, '밑반찬', '볶음', '일상', <oracledb.LOB object at 0x0000019A7BB93250>, None)
('http://www.10000recipe.com/recipe/4589585', '집에서도 근사하게 스테이크 :)', '1인분', '30분 이내', '초급', "{'재료': ['부채살2덩이', '브로콜리', '버섯', '당근', '양파'], '양념': ['홀그레인머스타드']}", '[]', 11, 50000, '양식', '굽기', '일상', <oracledb.LOB object at 0x0000019A7BB93310>, None)
('http://www.10000recipe.com/recipe/1519352', '상큼한 레몬 ..공부로 지친 우리아이들 간식만들기', '1인분', '60분 이내', '중급', "{'재료': [

In [11]:
rows[0]

('http://www.10000recipe.com/recipe/6869920',
 '입맛돋궈주는 새콤아삭 콩나물냉채',
 '4인분',
 '15분 이내',
 '아무나',
 "{'재료': ['콩나물200g', '맛살4T', '당근조금', '양파1/4개', '오이(생략가능)1/2개'], '소스': ['설탕4.5T', '다진마늘0.5T', '간장2T', '식초5T', '매실액기스(올리고당대체가능)1T', '참기름쪼르륵', '깨솔솔', '겨자1.5T']}",
 "['입맛돋우는음식', '간단반찬', '새콤달콤반찬', '밑반찬', '콩나물냉채']",
 11,
 104000,
 '샐러드',
 '무침',
 '일상',
 None)

In [4]:
import pandas as pd
data = pd.read_csv(r'C:\Users\HwijunKwon\Downloads\RecipeData.csv')
toy = data[["recipe_title", "recipe_ingredients"]].iloc[:1000].copy()

In [5]:
# 1. 이상한 문자열 제거

toy["recipe_ingredients"] = toy["recipe_ingredients"].apply(lambda x: x.replace('\\ufeff', '').replace('\\u200b', ''))


In [6]:
# 2.식재료 칼럼 만들기 (20개)
for i in range(1, 21):
    toy.loc[:, f'ingredient{i}'] = None
    toy.loc[:, f'quantity{i}'] = None
    toy.loc[:, f'unit{i}'] = None


In [7]:
toy["recipe_ingredients"][1]

"{'재료': ['건새우50g', '물1/4컵', '마늘종300g', '소금1/3큰술'], '양념재료': ['설탕2큰술', '간장4큰술', '식용유4큰술', '참기름2큰술', '물엿1+1/2큰술', '통깨1큰술']}"

In [4]:
# 3. 식재료 칼럼 쪼개기
from tqdm import tqdm
import ast
import re

# 패턴과 일치하지 않는 데이터를 저장할 딕셔너리
non_matching_items = {}

for idx, row in tqdm(toy.iterrows(), total=toy.shape[0]): #tqdm으로 진행상황 확인
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])  # row["recipe_ingredients"] :{'재료': ['건새우50g', '물1/4컵', ...], '양념재료': ['설탕2큰술', '간장4큰술', ...}"
    ingredient_count = 1
    for category, items in ingredients_dict.items():
        if items:  # 아이템이 존재하는 경우
            for item in items:
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    toy.at[idx, f'ingredient{ingredient_count}'] = ingredient
                    toy.at[idx, f'quantity{ingredient_count}'] = quantity
                    toy.at[idx, f'unit{ingredient_count}'] = unit

                    ingredient_count += 1
                else:
                    # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                    non_matching_items[idx] = item

# 패턴과 일치하지 않는 데이터 출력
for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:04<00:00, 236.83it/s]

Row 258: (소주or맛술)1큰술
Row 367: 18배멸치육수2T
Row 598: 
Row 837: 
Row 892: 


In [9]:
toy.head(3)

,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit17,ingredient18,quantity18,unit18,ingredient19,quantity19,unit19,ingredient20,quantity20,unit20
0,비빔밥재료 애호박나물 애호박볶음 아삭거리게 볶는 방법,"{'재료': ['애호박1개', '다진마늘0.5큰술', '소금0.5큰술', '참기름1...",애호박,1,개,다진마늘,0.5,큰술,소금,0.5,...,None,None,None,None,None,None,None,None,None,None
1,[마늘종볶음]단짠단짠 자꾸만 손이가요~,"{'재료': ['건새우50g', '물1/4컵', '마늘종300g', '소금1/3큰술...",건새우,50,g,물,1/4,컵,마늘종,300,...,None,None,None,None,None,None,None,None,None,None
2,백종원의 부추 달걀 볶음,"{'재료': ['부추1줌', '달걀4개', '식용유2큰술', '밥2공기'], '양념...",부추,1,줌,달걀,4,개,식용유,2,...,None,None,None,None,None,None,None,None,None,None


In [16]:
# csv 파일로 저장
toy.to_csv("data/toy.csv", index=False)

In [10]:
# 식재료 단위 집합
combined_units = []
for pair in zip(toy['unit1'], toy['unit2']) :
    for unit in pair :
        combined_units.append(unit)

unique_units = list(set(combined_units))

In [11]:
from collections import Counter

unit_counts = Counter(combined_units)
for unit, count in unit_counts.items():
    print(f"{unit}: {count}번")

개: 625번
큰술: 94번
g: 379번
컵: 38번
줌: 61번
줄: 11번
None: 219번
통: 5번
공기: 7번
봉지: 31번
모: 7번
그람: 6번
T: 60번
Ts: 6번
팩: 32번
주먹: 13번
t: 8번
대접: 5번
스푼: 37번
쪽: 8번
수저: 8번
개정도: 7번
g정도: 2번
TS: 4번
조각: 4번
알: 8번
대: 19번
g가량: 1번
티스푼: 1번
송이: 14번
줄기: 4번
장: 114번
그램: 9번
웅큼: 3번
캔: 11번
숟갈: 6번
봉: 16번
바구니: 1번
마리: 10번
천원어치: 1번
덩이: 1번
근: 2번
사발: 1번
소주컵: 1번
단: 7번
밥숟가락: 2번
뿌리: 7번
통약간더: 1번
토막: 3번
묶음: 3번
개반: 2번
밥숟갈: 2번
숟가락: 9번
줌정도: 2번
알정도: 1번
장분량정도: 1번
포기: 10번
꼬집: 4번
개또는반대: 1번
넉넉히: 1번
크술: 1번
작은술: 4번
잔: 1번
ml: 3번
타래: 1번
g간장: 1번
대접정도: 1번
접시: 1번
봉or: 1번
개작으면: 1번
소고기: 1번
ts: 1번
개작은것은: 1번
분의: 1번
cm: 2번
톨: 1번
g한봉지: 1번
움큼: 1번
종이컵: 6번
덩어리: 1번
kg: 1번
G: 1번
주먹정도: 1번
줌반: 1번
그릇: 3번
키로: 2번
식용유: 1번
큰국그릇: 1번
큰컵: 1번
cm길이: 1번
